<a href="https://colab.research.google.com/github/jude1997felix/MNIST-Classification/blob/main/MNIST_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [69]:
import numpy as np
from keras.models import model_from_json
import pandas as pd

from tensorflow import keras
from tensorflow.keras import layers

In [70]:
#  Model / data parameters
num_classes = 10
input_shape = (28, 28, 1)

path = '/content/trainset.csv'

df = pd.read_csv(path)
n = len(df)
train = df[0:int(n*0.85)]
test = df[int(n*0.85):]
len(train)

23800

In [71]:
y_train = train['label'].values
del train['label']
x_train = train.values

y_test = test['label'].values
del test['label']
x_test = test.values


In [72]:
x_train = x_train.reshape(len(train),28,28)
x_test = x_test.reshape(len(test),28,28,1)
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)

print("x_train shape:", x_train.shape)
print(x_train.shape[0], "train samples")
print(x_test.shape[0], "test samples")

x_train shape: (23800, 28, 28, 1)
23800 train samples
4200 test samples


In [73]:
# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
print(y_train.shape)

(23800, 10)


In [74]:
model = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation="softmax"),
    ]
)

model.summary()


Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_8 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
flatten_4 (Flatten)          (None, 1600)              0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 1600)              0         
_________________________________________________________________
dense_4 (Dense)              (None, 10)               

In [75]:
batch_size = 128
epochs = 25

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)

Epoch 1/25
168/168 [==============================] - 1s 6ms/step - loss: 4.2781 - accuracy: 0.4062 - val_loss: 0.7278 - val_accuracy: 0.7685
Epoch 2/25
168/168 [==============================] - 1s 5ms/step - loss: 0.7865 - accuracy: 0.7381 - val_loss: 0.3608 - val_accuracy: 0.8937
Epoch 3/25
168/168 [==============================] - 1s 5ms/step - loss: 0.5044 - accuracy: 0.8366 - val_loss: 0.2445 - val_accuracy: 0.9307
Epoch 4/25
168/168 [==============================] - 1s 5ms/step - loss: 0.3888 - accuracy: 0.8747 - val_loss: 0.2065 - val_accuracy: 0.9429
Epoch 5/25
168/168 [==============================] - 1s 5ms/step - loss: 0.3330 - accuracy: 0.8923 - val_loss: 0.1824 - val_accuracy: 0.9462
Epoch 6/25
168/168 [==============================] - 1s 5ms/step - loss: 0.2920 - accuracy: 0.9067 - val_loss: 0.1658 - val_accuracy: 0.9546
Epoch 7/25
168/168 [==============================] - 1s 5ms/step - loss: 0.2642 - accuracy: 0.9131 - val_loss: 0.1744 - val_accuracy: 0.9458
Epoch 

In [76]:
score = model.evaluate(x_test, y_test, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

Test loss: 0.09785305708646774
Test accuracy: 0.970714271068573


In [77]:
test_path = '/content/testset.csv'
raw_test_data = pd.read_csv(test_path)

test_data = raw_test_data.values
test_data = test_data.reshape(len(test_data),28,28)
test_data = np.expand_dims(test_data, -1)
print(test_data.shape)

(14000, 28, 28, 1)


In [78]:
test_values = model.predict_classes(test_data)

In [79]:
test_values = pd.DataFrame(test_values,columns=['Label'])

imageID = pd.DataFrame(range(14000),columns=['ImageID'])
imageID.index = range(14000)
imageID['ImageID'] = imageID['ImageID']+1

test_submission = imageID.join(test_values)
test_submission.head()

,ImageID,Label
0,1,3
1,2,1
2,3,3
3,4,8
4,5,0


In [80]:
test_submission.to_csv('test_values.csv',index=False)